In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow import keras

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from imblearn.over_sampling import *

In [13]:
df = pd.read_csv("winequality-red.csv")
y_tmp = np.array(df[["quality"]])
y = []
for u in range(0,np.shape(y_tmp)[0]):
    #print(y[u])
    if y_tmp[u] < 5:
        y =np.concatenate((y, [0]))
    elif y_tmp[u] < 7:
        y =np.concatenate((y, [1]))
    else:
        #print("="*5)
        y =np.concatenate((y, [2]))
        
print(y)
        
df2 = df.drop(["quality"], axis=1)
X = np.array(df2)
print("Original datashape: {}".format(np.shape(X)))
print("Original datashape: {}".format(np.shape(y_tmp)))

ovs = RandomOverSampler()
Xres, yres = ovs.fit_resample(X, y)

good = 0
mid = 0
bad = 0
rest = 0
for e in yres:
    if e == 0:
        bad = bad + 1
    elif e == 1:
        mid = mid +1
    elif e == 2:
        good = good +1
    else:
        print(e)
        rest = rest +1
print(good)
print(mid)
print(bad)
print(rest)

scaler = StandardScaler()
scaler.fit(Xres)
Xres = scaler.transform(Xres)

X_train, X_test, y_train, y_test = train_test_split( Xres, yres, test_size=0.1, random_state=42)
y_train = to_categorical(y_train)
X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size=0.01, random_state=42)
print("Upsampled datashape: {}".format(np.shape(Xres)))
print("Upsampled datashape: {}".format(np.shape(yres)))
print(yres)

[1. 1. 1. ... 1. 1. 1.]
Original datashape: (1599, 11)
Original datashape: (1599, 1)
1319
1319
1319
0
Upsampled datashape: (3957, 11)
Upsampled datashape: (3957,)
[1. 1. 1. ... 2. 2. 2.]


In [16]:
model = keras.Sequential(
    [
        layers.Dense(11, activation="relu", name="layer1"),
        layers.Dense(64, activation="relu", name="layer2"),
        layers.Dense(128, activation="relu", name="layer3"),
        layers.Dense(32, activation="relu", name="layer4"),
        layers.Dense(3, activation="softmax" , name="out"),
    ]
)
model.compile(optimizer=keras.optimizers.Adam(), loss="categorical_crossentropy")

In [17]:
history = model.fit(X_train, y_train,
                    batch_size=5,
                    epochs=50,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(X_val, y_val)
                   )

print('\nhistory dict:', history.history)

Train on 3525 samples, validate on 36 samples
Epoch 1/50
3525/3525 [==============================] - 1s 349us/sample - loss: 0.6905 - val_loss: 0.8894
Epoch 2/50
3525/3525 [==============================] - 1s 234us/sample - loss: 0.5192 - val_loss: 0.8023
Epoch 3/50
3525/3525 [==============================] - 1s 239us/sample - loss: 0.4517 - val_loss: 0.5898
Epoch 4/50
3525/3525 [==============================] - 1s 229us/sample - loss: 0.3920 - val_loss: 0.5717
Epoch 5/50
3525/3525 [==============================] - 1s 192us/sample - loss: 0.3741 - val_loss: 0.5719
Epoch 6/50
3525/3525 [==============================] - 1s 204us/sample - loss: 0.3415 - val_loss: 0.5182
Epoch 7/50
3525/3525 [==============================] - 1s 242us/sample - loss: 0.3148 - val_loss: 0.4575
Epoch 8/50
3525/3525 [==============================] - 1s 274us/sample - loss: 0.3043 - val_loss: 0.4318
Epoch 9/50
3525/3525 [==============================] - 1s 223us/sample - loss: 0.2632 - val_loss: 0.4168


In [18]:
train_predictions=model.predict(X_test)
y_pred = []
for t in train_predictions:
    y_pred.append(np.argmax(t))
acc = accuracy_score(y_test,y_pred)
print("Accuracy: {:.4%}".format(acc))

Accuracy: 95.2020%
